# Rephrasing Experiments

## Setup

In [1]:
# %pip install torch
# %pip install peft
# %pip install bitsandbytes

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftConfig, PeftModel

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
device

device(type='cpu')

In [3]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   # bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
messages = [
'''I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎''',
'''I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎''',
'''I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎''',
'''Your diary looks empty for this period 🤔''',
'''come back when you put some data in your diary ☺️''',
'''Ok, let me analyse your data some more... 🔎''',
'''Okay, will be back soon with more info... 🔎''',
'''(Advanced analysis requires time, I'll be back when I'm done...)''',
'''(This may take some time, I'll send you a message when it's done...)''',
'''⚠ WARNING:  you're comparing two periods with different lenghts. The comparison might be unsafe.''',
'''Some days at the beginning and/or end are missing. 🧐 
 
 I removed these for you: 
 - Mar 01 - Mar 05''',
'''⚠ FOOD: In the chart below you can see how different foods contributes to your intake:''',
'''⚠ TREND: you should get more carbohydrates, but you're slightly decreasing it .
 It would be better to revert this. 🧐 ''',
'''⚠ CONSISTENCY: Your daily intake seems pretty variable: try to eat similar amounts every day. 🧐''',
'''✅ TREND: you should get more sodium, in fact you're lightly increasing it .
 This is the right way to go. ☺️ 
 
 ⚠ CONSISTENCY: I see high variability in your daily intake: you should try to avoid changing it so often 🧐''',
'''⚠ TREND: you should keep your actual energy intake, instead you're slightly decreasing it .
 This is ok for now but better stabilise it to preserve this result 🤔 
 📈 PROGRESS: The trend is about the same as before 🤔 
 
 ✅ CONSISTENCY: Your daily intake looks regular, good job! 😁 
 📈 PROGRESS: The consistency didn't really change 🤔''',
'''🍖 PROTEIN 🍖 
 
 ⚠ ON AVERAGE: You had 83% (17% deficit). It should be more, we'll have to work on it. 🧐
 
 
 ⚠ TOUGHEST DAY: April 28, your intake was 52% (48% missing). It's not enough, we have to address this. 🧐''',
'''🍬 SUGAR 🍬 
 
 ⚠ ON AVERAGE: You got 55% (45% missing). You need to increase it, we have to address this. 🤔
 
 
 ⚠ TOUGHEST DAY: April 26, your intake was 32% (68% missing). It's not enough, we have to address this. 🤔''',
'''🔥 ENERGY 🔥 
 
 ✅ ON AVERAGE: I see a good balance in your energy intake. Well done! 😃 
 📈 PROGRESS: The average energy intake is about the same as before 🧐 
 
 ⚠ TOUGHEST DAY: April 17, your intake was 51% (49% missing). It's not enough, we have to address this. 🧐
 📈 PROGRESS: The intake for most off-plan day is about the same as before 🧐''',
]

In [27]:
REPHRASING_PROMPT = "Rephrase the following message to the user, keeping any mentioned dates. Do not introduce new dates or assume time periods. Do not add extra information. Use emojis."

In [6]:
def generate_output(messages, model, tokenizer):
    for message in messages:
        print('input:', message)
        context = [
            {"role": "system", "content": REPHRASING_PROMPT},
            {"role": "user", "content": message},
        ]

        input_ids = tokenizer.apply_chat_template(
            context,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        response = outputs[0][input_ids.shape[-1]:]
        print('output:', tokenizer.decode(response, skip_special_tokens=True))
        
        print()

In [7]:
import requests
import json

def generate_ollama_output(model, message):
    print('input:', message)

    url = 'http://localhost:11434/api/generate'
    payload = {
        "model": model,
        "system": REPHRASING_PROMPT,
        "prompt": message,
        "options": {
            "temperature": 0.6
        },
        "stream": False
    }
    payload_json = json.dumps(payload, ensure_ascii=False)
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=payload_json.encode('utf-8'), headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        output = json.loads(response.content.decode('utf-8'))['response']
        print('output:', output)
        print()
    else:
        print("Error in rephrasing request:", response.status_code, 'response:', response.content.decode('utf-8'))

In [8]:
url = 'http://localhost:11434/api/generate'
payload_json = json.dumps({"model": "llama3"})
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=payload_json.encode('utf-8'), headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    print('llama3 loaded into memory:', response.content.decode('utf-8'))
else:
    print("Error in rephrasing request:", response.status_code, 'response:', response.content.decode('utf-8'))

llama3 loaded into memory: {"model":"llama3","created_at":"2025-05-16T08:53:16.653140204Z","response":"","done":true,"done_reason":"load"}


## Llama 3

In [ ]:
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

### Full model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16)

In [ ]:
generate_output(messages, model, tokenizer) # 9:20

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: 📊 I'm analyzing your data from April 27, 2024... 🔎

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: 📊 I'm analyzing your data from Apr 22 2024 to Apr 28 2024... 🔎

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output: 📊 I'm reviewing your data from March 1st, 2024 to March 31st, 2024 and from April 1st, 2024 to April 30th, 2024. 🔎

input: Your diary looks empty for this period 🤔
output: 📅 Your diary looks empty for this period 🤔

input: come back when you put some data in your diary ☺️
output: Come back when you fill in your diary 📅☺️

input: Ok, let me analyse your data some more... 🔎
output: 🔍 I'm taking a closer look at your data... 🔎

input: Okay, will be back soon with more info... 🔎
output: 👋 I'll be back soon with more info..

### 4-bit Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:29<00:00,  7.33s/it]


In [ ]:
                #     print('input:', output)
                #     context = [
                #         {"role": "system", "content": REPHRASING_PROMPT},
                #         {"role": "user", "content": output},
                #     ]

                #     input_ids = rephrasing_tokenizer.apply_chat_template(
                #         context,
                #         add_generation_prompt=True,
                #         return_tensors="pt"
                #     ).to(rephrasing_model.device)

                #     terminators = [
                #         rephrasing_tokenizer.eos_token_id,
                #         rephrasing_tokenizer.convert_tokens_to_ids("<|eot_id|>")
                #     ]

                #     outputs = rephrasing_model.generate(
                #         input_ids,
                #         max_new_tokens=256,
                #         eos_token_id=terminators,
                #         do_sample=True,
                #         temperature=0.6,
                #         top_p=0.9,
                #     )
                #     response = outputs[0][input_ids.shape[-1]:]
                #     output = rephrasing_tokenizer.decode(response, skip_special_tokens=True)

                #     # use message in quotes if quotes present
                #     if '"' in output:
                #         open_quote_pos = output.index('"')
                #         if '"' in output[open_quote_pos:]:
                #             end_quote_pos = output[open_quote_pos:].index('"')
                #             output = output[open_quote_pos:end_quote_pos]

                #     print('output:', output)

In [ ]:
generate_output(messages, model, tokenizer) # 2:00

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: 🔎 I'm analyzing your data from April 27, 2024...

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: 🔎 I'm analyzing your data from April 22, 2024 to April 28, 2024...

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output: 📊 I'm reviewing your data from March 1st, 2024 to March 31st, 2024 and April 1st, 2024 to April 30th, 2024. 🔎

input: Your diary looks empty for this period 🤔
output: Your diary looks empty for now 🤔

input: come back when you put some data in your diary ☺️
output: 📆 Come back when you've filled in your diary! ☺️

input: Ok, let me analyse your data some more... 🔎
output: 🔍 I'm still reviewing your information...

input: Okay, will be back soon with more info... 🔎
output: 👋 See you soon! 🔎

input: (Advanced analysis 

### 4-bit, Static kv-cache and torch.compile

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16)
model.generation_config.cache_implementation = "static"
model = torch.compile(model, mode="reduce-overhead", fullgraph=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:44<00:00, 11.25s/it]


In [ ]:
def generate_output(messages, model, tokenizer):
    for message in messages:
        print('input:', message)
        context = [
            {"role": "system", "content": REPHRASING_PROMPT},
            {"role": "user", "content": message},
        ]

        input_ids = tokenizer.apply_chat_template(
            context,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        response = outputs[0][input_ids.shape[-1]:]
        print('output:', tokenizer.decode(response, skip_special_tokens=True))
        
        print()

In [ ]:
generate_output(messages, model, tokenizer)

### 4-bit and Flash Attention

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    load_in_4bit=True,
    attn_implementation="flash_attention_2",
)

In [ ]:
generate_output(messages, model, tokenizer) # 2:00

### Ollama

In [28]:
messages = [
# counselling
# ('Would you like some help with a struggle?', 'counselling'),
# ('Hmm... Would you like to try explaining again? It can help to give some extra sentences to describe your problem!', 'retry_counselling'),
# ('Was this helpful?', 'ask_feedback'),

# prompting
# ("Hey! I noticed that your food diary from yesterday is still empty... It's important to log your foods while you still remember what you've eaten!", 'check_diary'),
# ("Hey! I noticed a food entry with more than 1 kilogram in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed a food entry with more than 2 litres in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed a food entry with more than 4 cups in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed a food entry with more than half your calorie goal in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed less than 4 food entries in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed less than half of your calorie goal in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),
# ("Hey! I noticed more than twice your calorie goal in your food diary yesterday... Maybe double check to see if everything's logged correctly?", 'check_diary'),

# no_dates
# ('*****no_dates*****', False),
# ('''My apologies, but I don't get what time you asked for 😕''', 'no_dates'),
# ('''For dates, this is the format I understand:
#  - day: numbers (0-31)
#  - months: the name either full or abbreviated (e.g.: January or Jan)
#  - year: complete year (e.g.: 2005)

# #  For example I'll understand: Jan 10 2005! 😊''', 'no_dates'),

# no_multiple_times
# ('*****no_multiple_times*****', False),
# ('''That's more than one time period, I can't do that. 🧐''', 'no_multiple_times'),
# ('''I can't provide insights for multiple time periods 🧐''', 'no_multiple_times'),
# ('''But I can provide comparisons! Try something like "Compare today with yesterday" ☺️''', 'no_multiple_times'),
# ('''Perhaps you're looking for comparisons, like "Was this week better than last week"? 😄''', 'no_multiple_times'),

# wait
# ('*****wait*****', False),
# ('''I'm checking your data: 
 
#  - from Apr 27 2024 
 
#  and getting the insights... 🔎''', 'wait'),
# ('''I'm checking your data: 
 
#  - from Apr 22 2024 to Apr 28 2024 
 
#  and getting the insights... 🔎''', 'wait'),
# ('''I'm analysing your data: 
 
#  - from Mar 01 2024 to Mar 31 2024 
#  - and from Apr 01 2024 to Apr 30 2024 
 
#  and checking what changed... 🔎''', 'wait'),
# ('''I'm analysing your data:

#  - about *energy*
#  - *from Apr 22 2024 to Apr 28 2024*

#  and getting the insights... 🔎''', 'wait'),
# ('''I'm analysing your data:

#  - about *sodium*
#  - *from Apr 22 2024 to Apr 28 2024*

#  and getting the insights... 🔎''', 'wait'),

# wait_more
# ('*****wait_more*****', False),
# ('''Ok, let me analyse your data some more... 🔎''', 'wait_more'),
# ('''Okay, will be back soon with more info... 🔎''', 'wait_more'),
# ('''(Advanced analysis requires time, I'll be back when I'm done...)''', 'wait_more'),
# ('''(This may take some time, I'll send you a message when it's done...)''', 'wait_more'),

# empty
('*****empty*****', False),
('''Your diary looks empty for this period 🤔''', 'empty'),
('''I see...no data this period 🧐''', 'empty'),
('''come back when you put some data in your diary ☺️''', 'empty'),
('''I can't help you until you've filled your diary ☺️''', 'empty'),

# holes_inside_warn
('*****holes_inside_warn*****', False),
('''Some days are missing. I can estimate them but the final value could be different to reality. Would you like me to estimate your data?''', 'holes_inside_warn'),
('''⚠ *ATTENTION: SOME DATA ARE ESTIMATED* ⚠
 *You've made me *approximate some data* because they were missing. The values you see *might be less accurate*.*
 Also, you won't be able to view advanced insights, as this wouldn't be safe for you.''', 'holes_inside_warn'),

# holes_surrounding_warn
('*****holes_surrounding_warn*****', False),
('''Some days at the beginning and/or end are missing. 🧐 
 
 I removed these for you: 
 - Mar 01 - Mar 05''', 'holes_surrounding_warn'),

# most_data_missing
('*****most_data_missing*****', False),
('''Most of the days in that period are empty, so I'm sorry to say that I can't do that. 😔''', 'most_data_missing'),
('''Go fill the empty days and try again 😁''', 'most_data_missing'),

# query_complexity_excess
('*****query_complexity_excess*****', False),
('''That's too hard for me, my time limit is 5 months 😅''', 'query_complexity_excess'),

# update
('*****update*****', False, False),
('''🔥 *ENERGY* 🔥

 ⚠ *ON AVERAGE:* You had *61% (39% missing)*. You need to increase it, we'll have to work on it. 🧐


 ⚠ *TOUGHEST DAY:* I'm sorry but with only one day I can't tell you this 😕''', 'update', 'energy'),
('''🔥 *ENERGY* 🔥

 ✅ *ON AVERAGE:* I see a good balance in your energy intake. That's great! 😊

 ⚠ *TOUGHEST DAY:* April 17, your intake was *51% (49% missing)*. It needs to be higher, we'll have to work on it. 🧐''', 'update', 'energy'),
('''⚠ *TREND:* you should keep your actual energy intake, unfortunately you're lightly reducing it .
 This is ok for now but try stabilising this or it could affect your diet 🤨

 ✅ *CONSISTENCY:* Your daily intake looks regular, nice work! 🙂''', 'update', 'energy'),
('''⚠ FOOD: In the chart below you can see how different foods contributes to your intake:''', 'update', 'energy'),
('''⚠ *TREND AND CONSISTENCY:* sorry, I need 3 or more days for this 😕''', 'update', 'carbohydrates'),
('''⚠ TREND: you should get more carbohydrates, but you're slightly decreasing it .
 It would be better to revert this. 🧐 
 
 ⚠ CONSISTENCY: Your daily intake seems pretty variable: try to eat similar amounts every day. 🧐''', 'update', 'carbohydrates'),
('''✅ TREND: you should get more sodium, in fact you're lightly increasing it .
 This is the right way to go. ☺️ 
 
 ⚠ CONSISTENCY: I see high variability in your daily intake: you should try to avoid changing it so often 🧐''', 'update', 'sodium'),
('''🍖 PROTEIN 🍖 
 
 ⚠ ON AVERAGE: You had 83% (17% deficit). It should be more, we'll have to work on it. 🧐
 
 
 ⚠ TOUGHEST DAY: April 28, your intake was 52% (48% missing). It's not enough, we have to address this. 🧐''', 'update', 'protein'),
('''🍬 SUGAR 🍬 
 
 ⚠ ON AVERAGE: You got 55% (45% missing). You need to increase it, we have to address this. 🤔
 
 
 ⚠ TOUGHEST DAY: April 26, your intake was 32% (68% missing). It's not enough, we have to address this. 🤔''', 'update', 'sugar'),

# filter_more_info_button
('*****filter_more_info_button*****', False),
('''🔥 ENERGY 🔥
 What do you want to know? 🤔

 Tap anything you need (tap again to cancel) and confirm when ready 😄''', 'filter_more_info_button'),
('''🧈 FAT 🧈
 What do you want to know? 🤔''', 'filter_more_info_button'),

# empty_message_stack
('*****empty_message_stack*****', False),
('''Sure, let's stop here 🙂''', 'empty_message_stack'),
('''3,2,1....all gone!''', 'empty_message_stack'),
('''Ok, let\'s forget about that''', 'empty_message_stack'),

# weird_comparison
('*****weird_comparison*****', False),
('''⚠ WARNING:  you're comparing two periods with different lengths. The comparison might be unsafe.''', 'weird_comparison'),
('''⚠ *WARNING: * these periods have some *days in common*. The comparison might be unsafe.''', 'weird_comparison'),
('''⚠ *WARNING: * you *cut out some days* between the two periods. The comparison might be unsafe.''', 'weird_comparison'),

# compare
('*****compare*****', False, False),
('''⚠ TREND: you should keep your actual energy intake, instead you're slightly decreasing it .
 This is ok for now but better stabilise it to preserve this result 🤔 
 📈 PROGRESS: The trend is about the same as before 🤔 
 
 ✅ CONSISTENCY: Your daily intake looks regular, good job! 😁 
 📈 PROGRESS: The consistency didn't really change 🤔''', 'compare', 'energy'),
('''🔥 ENERGY 🔥 
 
 ✅ ON AVERAGE: I see a good balance in your energy intake. Well done! 😃 
 📈 PROGRESS: The average energy intake is about the same as before 🧐 
 
 ⚠ TOUGHEST DAY: April 17, your intake was 51% (49% missing). It's not enough, we have to address this. 🧐
 📈 PROGRESS: The intake for most off-plan day is about the same as before 🧐''', 'compare', 'energy'),

# compare no dates
('Please give me two dates or a date range to compare', 'compare_no_dates'),
('''It looks like you've only given me one date or date range! Could you give me two to compare?''', 'compare_one_date'),
('''I'm sorry, but I can only compare two dates or date ranges''', 'compare_many_dates'),
]

In [ ]:
messages = [
    ('''🔥 *ENERGY* 🔥

 ⚠ *ON AVERAGE:* Your intake was *15% more than your goal*. It needs to be lower, we have to address this. 🤨


 ⚠ *TOUGHEST DAY:* sorry, I need at least two days to get this 😕''', 'update', 'energy'),
    ('''⚠ *TREND AND CONSISTENCY:* for this I need at least 3 days, sorry 😕''', 'update', 'sugar'),
    ('''⚠ *FOOD*: In the below chart you can see how different foods contribute to your intake:''', 'update', 'sodium'),
    ('''🍝 *CARBOHYDRATES* 🍝

 ⚠ *ON AVERAGE:* You got *67% (33% missing)*. It's not enough, we have to address this. 🧐


 ⚠ *TOUGHEST DAY:* sorry, I need at least two days to get this 😕''', 'update', 'carbohydrates'),
    ('''🍬 *SUGAR* 🍬

 ✅ *ON AVERAGE:* I see a good balance in your sugar intake. That's great! 😎

 ⚠ *TOUGHEST DAY:* May 05, your intake was *73% (27% missing)*. You need to increase it, we have to address this. 🧐''', 'update', 'sugar'),
    ('''⚠ *TREND:* you should keep your actual energy intake, instead you're lightly reducing it .
 It's ok for now but try stabilising this or it could affect your diet 🤨

 ⚠ *CONSISTENCY:* Your daily intake seems pretty variable: you should try to regularise it. 🤨''', 'update', 'energy'),
    ('''✅ *FOOD*: In the chart below you can see how different foods contribute to your intake:''', 'update', 'energy'),
    ('''🧈 *FAT* 🧈

 ⚠ *ON AVERAGE:* You got *84% (16% deficit)*. It needs to be higher, we have to address this. 🤨
 📈 *PROGRESS:* The average fat intake didn't really change 🤨

 ⚠ *TOUGHEST DAY:* April 17, your intake was *51% (49% missing)*. It needs to be higher, we have to address this. 🤨
 📈 *PROGRESS:* The intake for most off-plan day looked better before (now it's 14% worse). 😕''', 'compare', 'fat'),
    ('''✅ *TREND:* you should get more energy, in fact, you're slightly rising it .
 You should keep this up. 😃

 📈 *PROGRESS:* The trend looked better before 😕

 ✅ *CONSISTENCY:* Your data show a stable daily intake, that's great! 😃
 📈 *PROGRESS*: The consistency is about the same as before 🧐''', 'compare', 'energy'),
    ('''🍖 *PROTEIN* 🍖

 ⚠ *ON AVERAGE:* You had *70% (30% missing)*. You should increase your consumption, we need to work on this. 🤔
 📈 *PROGRESS:* The average protein intake improved (now it's 19% better). 😎

 ⚠ *TOUGHEST DAY:* sorry, if any of your periods has only one day I can't get this for you 😕''', 'compare', 'protein'),
    ('''⚠ *TREND AND CONSISTENCY:* for this I need at least 3 days for both periods, sorry 😕''', 'compare', 'sodium')
    
]

In [ ]:
REPHRASING_PROMPT = "Rephrase the following message to the user, keeping any mentioned numbers and dates. Do not introduce new dates. Do not assume unspecified time periods. Do not add extra information. Do not ask questions. Use emojis."

In [ ]:
REPHRASING_PROMPT = "Do not include additional information. Use simple language and emojis. Rephrase the following message to the user."

In [29]:
import requests
import json

def generate_ollama_output(model, prompt):#, message=None):
    # print('input:', message)
    print(prompt)

    url = 'http://localhost:11434/api/generate'
    payload = {
        "model": model,
        # "system": prompt,
        "prompt": prompt, #message
        "options": {
            "temperature": 0.6
        },
        "keep_alive": -1,
        "stream": False
    }
    payload_json = json.dumps(payload, ensure_ascii=False)
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=payload_json.encode('utf-8'), headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        output = json.loads(response.content.decode('utf-8'))['response']
        # print('output:', output)
        print(output)
        print()
    else:
        print("Error in rephrasing request:", response.status_code, 'response:', response.content.decode('utf-8'))

In [30]:
import requests
import json

def generate_ollama_output(model, prompt, message=None):
    print('input:', message)
    # print(prompt)

    url = 'http://localhost:11434/api/generate'
    payload = {
        "model": model,
        "system": prompt,
        "prompt": message,
        "options": {
            "temperature": 0.6
        },
        "keep_alive": -1,
        "stream": False
    }
    payload_json = json.dumps(payload, ensure_ascii=False)
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=payload_json.encode('utf-8'), headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        output = json.loads(response.content.decode('utf-8'))['response']
        # print('output:', output)
        print(output)
        print()
    else:
        print("Error in rephrasing request:", response.status_code, 'response:', response.content.decode('utf-8'))

In [ ]:
contexts = {
        'ask_feedback': 'You are asking for feedback on the supportive texts you provided the user with. ',
        'check_diary': "You are checking if the user made an error while filling in their food diary yesterday. Tell the user exactly what you noticed. ",
        'compare': 'The user requested comparative insights into their {} intake between two specified periods. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'compare_average': "The user requested comparative insights into their average {} intake and toughest day between two specified periods. Specified periods must be longer than one day to calculate toughest day. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ",
        'compare_trend': 'The user requested comparative insights into the trends and consistency of their {} intake between two specified periods. This can only be calculated if they specify periods both longer than 3 days. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'compare_food': 'The user requested comparative insights into the food items that contributed to their {} intake between two specified periods. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'compare_no_dates': 'The user requested comparative insights into their food diary but did not give dates. Do not greet the user. ',
        'compare_one_date': 'The user requested comparative insights into their food diary but only gave one date. Do not greet the user. ',
        'compare_many_dates': 'The user requested comparative insights into their food diary but gave too many dates. Do not greet the user. ',
        'counselling': 'You are asking whether the user needs helps with their diet. ',
        'retry_counselling': "You are asking the user whether they want to try explaining their diet issue again with different words. ",
        'empty': "The user requested insights into their food diary for a time period with no data. ",
        'empty_message_stack': 'The user has chosen to end the flow of information and buttons. ',
        'filter_more_info_button': 'The user is offered button options below for different types of advanced insights of their food diary. Keep the heading. Do not assume the options. ',
        'holes_inside_warn': "The user's food diary has empty days in the specified period, so insights will be calculated without the empty days. Mention any removed dates but do not assume the time period. ",
        'holes_surrounding_warn': "The user's food diary has empty days in the specified period, so insights will be calculated without the empty days. Mention any removed dates but do not assume the time period. Do not ask questions. ",
        'invalid_button': 'The user has clicked on a previous button for more advanced insights that is now outdated. You cannot retrieve this information. Do not apologise. ',
        'more_info_denied': "The user's food diary has estimated days that were empty in the specified period, so insights cannot be calculated. ",
        'most_data_missing': "The user's food diary has too many empty days in the specified period, so insights cannot be calculated. Do not ask questions. ",
        'no_dates': 'The user requested insights of their food diary but did not specify dates in the correct format. Do not include greetings. ',
        'no_dates_default': 'The user requested insights into their food diary but did not specify dates in the correct format. Do not include greetings. ',
        'no_multiple_times': 'The user either mentioned more than one time period for insights into their food diary, or more than two time periods for a comparison. Use the same examples. ',
        # 'partial_typo': '',
        'query_complexity_excess': 'The user requested insights into their food diary over a specified period longer than the maximum of 5 months. ',
        'update': 'The user requested insights into their {} intake over a specified period. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'update_average': "The user requested insights into their average {} intake and toughest day over a specified period. The specified period must be longer than one day to calculate toughest day. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ",
        'update_trend': 'The user requested insights into the trends and consistency of their {} intake over a specified period. This can only be calculated if they specify a period longer than 3 days. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'update_food': 'The user requested insights into the food items that contributed to their {} intake over a specified period. Keep all information including all percentages of daily goal. Do not greet the user or prompt a response. ',
        'wait': 'The user requested insights into their food diary for the specified period. Keep all mentioned dates. ',
        'wait_more': 'The user requested advanced insights into their food diary. ',
        'weird_comparison': 'The user requested comparative insights into their food diary across two time periods that overlap, are non-consecutive or are different in length. Do not ask questions. ',
    }

In [15]:
def format_instruction(prompt, message):
    return f"""{prompt}

### Message:
{message}

### Rephrased message:"""

In [ ]:
for message, intent, nutrient in messages:
    if intent==False:
        print(message)
        continue
    generate_ollama_output('llama3', format_instruction(contexts[intent].format(nutrient)+REPHRASING_PROMPT, message))
    print()

In [ ]:

for message, intent, nutrient in messages:
    
    if 'AVERAGE' in message:
        intent += '_average'
    elif 'TREND' in message:
        intent += '_trend'
    elif 'FOOD' in message:
        intent += '_food'
        
    if intent==False:
        print(message)
        continue
    generate_ollama_output('llama3', format_instruction(contexts[intent].format(nutrient)+REPHRASING_PROMPT, message))

### Unsloth Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    device_map="auto",
    torch_dtype=torch.bfloat16
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
generate_output(messages, model, tokenizer) # 10:40

In [ ]:
for message in messages:
    print('input:', message)
    input_text = '<start_of_turn>user ' + REPHRASING_PROMPT + message  + '<end_of_turn>\n<start_of_turn>model\nResponse:\n\n'
    input_ids = tokenizer(input_text, return_tensors="pt").to(device)

    output = model.generate(**input_ids, max_new_tokens=250, do_sample=True)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print('output:', decoded[len(input_text)-43:])
    
    print()

## Mistral Instruct

### Full Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards: 100%|██████████| 3/3 [01:03<00:00, 21.24s/it]


In [ ]:
for message in messages: # 9:50
    print('input:', message)
    context = [
        {"role": "user", "content": REPHRASING_PROMPT + '\n' + message},
        {"role": "assistant", "content": "Response:\n"},
    ]

    input_ids = tokenizer.apply_chat_template(
        context,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    print('output:', tokenizer.decode(response, skip_special_tokens=True))
    
    print()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: Great news! 😃 I'm currently analyzing your data from 📅 Apr 27, 2024. 🔎 Stay tuned for the insights!

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

output: мировая символика: Тренд: ваша реальная потребность в энергии немного уменьшается, а не сохраняется.
 Это в порядке дел в настоящее время, но лучше установите стабильность, чтобы сохранить это достижение 🤔 
 📈 Прогресс: Тренд примерно такой же, как раньше 🤔

 ✅ Постоянство: Ваша ежедневная потребность выглядит регулярной, хорошо сделано! 😁 
 📈 Прогресс: Постоянство не изменилось намного 🤔

input: 🍖 PROTEIN 🍖 
 
 ⚠ ON AVERAGE: You had 83% (17% deficit). It should be more, we'll have to work on it. 🧐
 
 
 ⚠ TOUGHEST DAY: April 28, your intake was 52% (48% missing). It's not enough, we have to address this. 🧐


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: 2. 🍔 **Low Protein Intake** 🍔 

🔨 **Average:** 📊 You consumed 83% of your daily protein requirement, falling short by 17%. Let's improve it! 🧐

🚨 **April 28:** 📅 Your protein intake was only 52%, meaning you missed the mark by 48%. Let's focus on increasing your intake that day! 🧐

input: 🍬 SUGAR 🍬 
 
 ⚠ ON AVERAGE: You got 55% (45% missing). You need to increase it, we have to address this. 🤔
 
 
 ⚠ TOUGHEST DAY: April 26, your intake was 32% (68% missing). It's not enough, we have to address this. 🤔


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: щ️ SWEETS ALERT 🍬 -----

❗️ AVERAGE: You've missed 45% of your intake 😕, let's work on improving it! 💡

❗️ TOUGHEST DAY: ☹️ April 26, you only consumed 32% 🙈, we need to boost that! 💪🏼

input: 🔥 ENERGY 🔥 
 
 ✅ ON AVERAGE: I see a good balance in your energy intake. Well done! 😃 
 📈 PROGRESS: The average energy intake is about the same as before 🧐 
 
 ⚠ TOUGHEST DAY: April 17, your intake was 51% (49% missing). It's not enough, we have to address this. 🧐
 📈 PROGRESS: The intake for most off-plan day is about the same as before 🧐
output: Great job keeping your energy intake in check 🔥

✅ AVERAGE: You're maintaining a good balance on average 😃
📈 PROGRESS: Your average energy intake remains similar to before 🧐

⚠️ WARNING: On April 17, your energy intake was only 51%, which is 49% less than required 🧐
📈 PROGRESS: The energy intake on your most off-plan day remains similar to before 🧐

💡 SUGGESTION: Let's work together to improve your intake on challenging days 💪🏼💡



### 4-bit Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.07s/it]


In [ ]:
for message in messages: # 3:30
    print('input:', message)
    context = [
        {"role": "user", "content": REPHRASING_PROMPT + '\n' + message},
        {"role": "assistant", "content": "Response:\n"},
    ]

    input_ids = tokenizer.apply_chat_template(
        context,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    print('output:', tokenizer.decode(response, skip_special_tokens=True))
    
    print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: great! 🌟 I'm currently analyzing your data from: 
- April 27, 2024 📅

Insights are on the way! 💡🔬

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

output: ми ⚠️ You're currently consuming less energy than before, 📉 but try to maintain your intake for consistent results, 🔄
👉 Your daily energy consumption remains consistent, keep up the good work, 👍🏽 💡
📈 Progress: The trend and consistency of your energy intake remain about the same, 🤔 📈

input: 🍖 PROTEIN 🍖 
 
 ⚠ ON AVERAGE: You had 83% (17% deficit). It should be more, we'll have to work on it. 🧐
 
 
 ⚠ TOUGHEST DAY: April 28, your intake was 52% (48% missing). It's not enough, we have to address this. 🧐


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: мированый\_бутерброд 🍔 \ud83c\udf0f PROTEIN \ud83c\udf0f ⚠️

\ud83d\ude2e You had 83% 📊 (17% deficit) \ud83d\ude2e. Needs improvement! 🧐

\ud83d\ude2e April 28: 52% 📊 (48% missing) \ud83d\ude2e. We need to work on this day. 🧐

input: 🍬 SUGAR 🍬 
 
 ⚠ ON AVERAGE: You got 55% (45% missing). You need to increase it, we have to address this. 🤔
 
 
 ⚠ TOUGHEST DAY: April 26, your intake was 32% (68% missing). It's not enough, we have to address this. 🤔


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


output: мироглификация: 🍬 SUGAR 🍬

❗️ АВГАРДИСХИД: Составил 55% (45% недостаточно). Нужно увеличить, нужно решать это. 🤔

❗️ САМЫЙ ТРЕБОВАТЕЛЬНЫЙ ДЕНЬ: К 26 апр. Ваш уровень потребления составлял 32% (68% недостаточно). Недостаточно, нужно решать это. 🤔

Translation:
Sweet emojis: 🍬 SUGAR 🍬

⚠️ AVERAGE: You had 55%, which is 45% less. You need to increase it, we have to address this. 🤔

⚠️ TOUGHEST DAY: On April 26, your intake was 32%, which is 68% less. It's not enough, we have to address

input: 🔥 ENERGY 🔥 
 
 ✅ ON AVERAGE: I see a good balance in your energy intake. Well done! 😃 
 📈 PROGRESS: The average energy intake is about the same as before 🧐 
 
 ⚠ TOUGHEST DAY: April 17, your intake was 51% (49% missing). It's not enough, we have to address this. 🧐
 📈 PROGRESS: The intake for most off-plan day is about the same as before 🧐
output: мирование Энергия/////////

✅ БАЛАНС: Твой баланс энергии в среднем впечатляет! 😊
📈 ПРОГРЕСС: Среднее потребление энергии примерно такое же, как ра

## Gemma

In [ ]:
REPHRASING_PROMPT = "Without assuming any dates but keeping numbers, rephrase the following message in a more natural way."

### Full Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:50<00:00, 12.65s/it]


In [ ]:
for message in messages: # 15:20
    print('input:', message)
    context = [
        {"role": "user", "content": REPHRASING_PROMPT + '\n' + message},
        {"role": "model", "content": "Response: "},
    ]

    input_ids = tokenizer.apply_chat_template(
        context,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    print('output:', tokenizer.decode(response, skip_special_tokens=True))
    
    print()

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: Sure, here is the rephrased message:

"I'm checking your data from **Apr 27 2024** and getting the insights... 🔎"

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: Sure, here is the rephrased message with the dates preserved:

Hey, I'm checking your data from **Apr 22 2024** to **Apr 28 2024** and getting the insights... 🔎

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output: Sure, here is the rephrased message with the mentioned dates preserved:

"I'm analyzing your data from Mar 01 2024 to Mar 31 2024 and from Apr 01 2024 to Apr 30 2024. I'm checking what changed... 🔎"

input: Your diary looks empty for this period 🤔
output: Sure, here's the rephrased message for the user:

Your diary is currently empty for this period 🤔.

input: 

### 4-bit Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:44<00:00, 11.21s/it]


In [ ]:
for message in messages: # 2:25
    print('input:', message)
    context = [
        {"role": "user", "content": REPHRASING_PROMPT + '\n' + message},
        {"role": "model", "content": "Response: "},
    ]

    input_ids = tokenizer.apply_chat_template(
        context,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    print('output:', tokenizer.decode(response, skip_special_tokens=True))
    
    print()

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: Sure, here is the rephrased message:

I'm checking your data from **Apr 27 2024** and getting the insights... 🔎

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: Sure, here is the rephrased message:

I'm checking your data from **Apr 22 2024** to **Apr 28 2024** and getting the insights... 🔎

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output: Sure, here's the rephrased message:

I'm analyzing your data:

- from **Mar 01 2024** to **Mar 31 2024**
- and from **Apr 01 2024** to **Apr 30 2024**

And checking what changed... 🔎

input: Your diary looks empty for this period 🤔
output: Sure, here is the rephrased message:

Your diary is currently empty for this period. 🤔 Is there anything you'd like to add?

input: come back when you put 

### 2b

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    # torch_dtype=torch.bfloat16
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.62s/it]


In [ ]:
for message in messages: # ??
    print('input:', message)
    context = [
        {"role": "user", "content": REPHRASING_PROMPT + '\n' + message},
    ]

    input_ids = tokenizer.apply_chat_template(
        context,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    print('output:', tokenizer.decode(response, skip_special_tokens=True))
    
    print()

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: Hey there! 👋

I'm taking a peek at your data from April 27, 2024. Let me analyze it and give you some insights! 🤔

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: Sure, here's the rephrased message:

🚨 Your data is being checked from April 22nd to April 28th, 2024! 🚨

Let us know what you find out! 👀

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output: Sure, here is the rephrased message:

🚨 Data Analysis: 🚨

- From Mar 01 to Mar 31, 2024 📢
- From Apr 01 to Apr 30, 2024 💥

I'm analyzing your data to see what changed! 🤔

input: Your diary looks empty for this period 🤔
output: Your diary looks empty right now 🥺. 🤔

input: come back when you put some data in your diary ☺️
output: 🤔 Your diary is waiting for some input! 📝 Give it some

### Ollama

In [ ]:
for message in messages:
    generate_ollama_output('gemma:7b', message)

input: I'm checking your data: 
 
 - from Apr 27 2024 
 
 and getting the insights... 🔎
output: Checking your data from Apr 27th 2024... 🔎

input: I'm checking your data: 
 
 - from Apr 22 2024 to Apr 28 2024 
 
 and getting the insights... 🔎
output: Checking data from April 22 to 28 🗓️ and gathering insights! 🔎

input: I'm analysing your data: 
 
 - from Mar 01 2024 to Mar 31 2024 
 - and from Apr 01 2024 to Apr 30 2024 
 
 and checking what changed... 🔎
output:  analizing your data from Mar 1st to Mar 31st and Apr 1st to Apr 30th 🗓️ to see what's changed! 🔎

input: Your diary looks empty for this period 🤔
output: Your diary is currently blank for this timeframe 📚🤔

input: come back when you put some data in your diary ☺️
output: Got data for your diary? 😉 Come back when you've added some!

input: Ok, let me analyse your data some more... 🔎
output: Analysing your data... ⏳

input: Okay, will be back soon with more info... 🔎
output: Be right back with more deets soon! 💻✨

input: (Advan